In [2]:
%load_ext dotenv
%dotenv

## Demo

For demo stration purpose, i only use the first 20 pages.

In [9]:
!pip install fitz

  Using cached configparser-7.0.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
Using cached configparser-7.0.0-py3-none-any.whl (16 kB)
Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 388.4 kB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 666.3 kB/s eta 0:00:001m2.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 1.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 1.7 MB/s eta 0:00:00m eta 0:00:010:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 655.5 kB/s eta 0:00:0031m2.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 2.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 33

In [11]:
! pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 730.0 kB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 2.7 MB/s eta 0:00:00m eta 0:00:010:00:01


In [12]:
from langfuse.decorators import observe
from langfuse.openai import openai # OpenAI integration
from langfuse import Langfuse
import os 
import fitz
from langchain.document_loaders import PyMuPDFLoader

In [13]:
langfuse = Langfuse(
  secret_key=os.environ.get("LANGFUSE_SECRET_KEY"),
  public_key=os.environ.get("LANGFUSE_PUBLIC_KEY"),
  host=os.environ.get("LANGFUSE_HOST")
)

In [23]:
PDF_PATH = "8708-chicken-soup-for-the-soul-tap-1-chia-se-tam-hon-va-qua-tang-cuoc-song-thuviensach.vn.pdf"

def extract_pages(pdf_path, no_pages=20):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    
    # Initialize a list to store the pages' text
    pages_text = []
    
    # Loop through the first 20 pages (or the total number of pages if less than 20)
    for page_num in range(min(no_pages, pdf_document.page_count)):
        page = pdf_document.load_page(page_num)
        pages_text.append(page.get_text())
    
    # Close the PDF file
    pdf_document.close()
    
    return pages_text

texts = extract_pages(PDF_PATH,20)

In [24]:
len(texts)

20

In [25]:
from langchain_openai import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain_core.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain_text_splitters import CharacterTextSplitter
from langchain import hub
import os
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain_community.document_loaders import ReadTheDocsLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

## Text Splitter

1. Create text splitter
2. Create the documents from list of string

We need to do that because the retrieval need to get page_content from each element.

In [26]:
# create text splitter

text_splitter = CharacterTextSplitter(chunk_size=600, chunk_overlap=50, separator="\n\n",)
texts = text_splitter.create_documents(texts)

In [27]:
texts

[Document(page_content='Cùng bạn đọc\nEbook miễn phí tại : www.Sachvui.Com\nSau khi thực hiện các tập Hạt Giống Tâm Hồn và những cuốn sách chia sẻ\nvề cuộc sống, First News đã nhận được sự đón nhận và đồng cảm sâu sắc\ncủa đông đảo bạn đọc khắp nơi. Chúng tôi nhận ra rằng sự chia sẻ về tâm\nhồn là một điều thực sự ý nghĩa trong cuộc sống hiện nay, giúp chúng ta\nvượt qua những nỗi buồn, những thất vọng để hướng đến một ngày mai tươi\nđẹp hơn với những ước mơ, hoài bão của mình và cảm nhận cuộc sống trọn\nvẹn hơn.\nTrên tinh thần đó, First News tiếp tục giới thiệu đến các bạn những câu\nchuyện hay được chọn lọc từ bộ sách Chicken Soup for the Soul nổi tiếng\ncủa hai tác giả Jack Canfield và Mark Victor Hansen.\nĐây là lần đầu tiên bộ sách Chicken Soup for the Soul được trình bày dưới\ndạng song ngữ để bạn đọc có thể cảm nhận được ý nghĩa câu chuyện bằng cả\nhai ngôn ngữ. Việc chuyển tải trọn vẹn và chính xác ý nghĩa sâu sắc của\nnhững câu chuyện theo nguyên bản tiếng Anh là một cố gắng 

## Vector store

Use FAISS as vector store


In [28]:
from langchain_pinecone import PineconeVectorStore

embeddings= OpenAIEmbeddings(model='text-embedding-ada-002', openai_api_key=os.environ.get("OPENAI_API_KEY"))
vectorstore= FAISS.from_documents(texts, embeddings)
vectorstore.save_local("book-summary-vectorstore")

## RAG

In [29]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
llm = ChatOpenAI(model='gpt-4o-mini')

combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
retrieval_chain = create_retrieval_chain(retriever=vectorstore.as_retriever(), combine_docs_chain=combine_docs_chain)

In [30]:
@observe()
def ask_bot(query):
    result = retrieval_chain.invoke({"input": query})
    return result['answer']

ask_bot("tác giả của cuốn sách là ai?")

'Tác giả của cuốn sách "Chicken Soup for the Soul" là Jack Canfield và Mark Victor Hansen.'

In [31]:
ask_bot("nội dung của chương 1")

'Xin lỗi, nhưng tôi không có thông tin cụ thể về nội dung của chương 1 từ văn bản mà bạn đã cung cấp. Nếu bạn cần biết về một chương cụ thể hoặc nội dung nào đó từ cuốn sách, bạn có thể tìm kiếm trong cuốn sách hoặc cung cấp thêm thông tin để tôi có thể hỗ trợ tốt hơn.'

In [32]:
ask_bot("Về tác giả và sự ra đời của Chicken Soup for the Soul")

'Tác giả của "Chicken Soup for the Soul" là Jack Canfield và Mark Victor Hansen, cả hai đều là những diễn thuyết gia nổi tiếng với nhiều buổi nói chuyện về cuộc sống, sức mạnh tinh thần và phát triển nội tâm. Jack Canfield là người tốt nghiệp Đại học Harvard và có nhiều kinh nghiệm trong việc truyền cảm hứng cho người khác, trong khi Mark Victor Hansen đã thực hiện hàng nghìn buổi nói chuyện trên toàn thế giới.\n\nQuyển sách "Chicken Soup for the Soul" ra đời từ mong muốn của thính giả sau những buổi diễn thuyết, họ muốn lưu giữ những câu chuyện có thật và ý nghĩa mà Jack và Mark đã chia sẻ. Vào năm 1990, hai tác giả bắt đầu thực hiện ý tưởng này, nhưng quá trình biên tập và tổng hợp các câu chuyện gặp nhiều khó khăn. Sau 3 năm làm việc, họ chỉ thu thập được 68 câu chuyện và quyết định hợp tác với các nhà diễn thuyết khác để hoàn thiện bộ sách.\n\nCuối cùng, vào ngày 28/6/1993, quyển sách đầu tiên đã được xuất bản và nhanh chóng trở thành một hiện tượng trong ngành xuất bản. Nó đã tạo 

In [33]:
print(ask_bot("số lượng sách đã bán là bao nhiêu ? "))

Số lượng sách đã bán là 85 triệu bản.


In [34]:
print(ask_bot("số tiền ủng hộ từ thiện "))

Số tiền ủng hộ từ thiện từ bộ sách "Chicken Soup for the Soul" là hơn 8.5 tỉ đô la.


In [35]:
print(ask_bot("trình bày về nội dung Love: The one creative force"))

Nội dung "Love: The one creative force" kể về một nghiên cứu xã hội học được thực hiện bởi một giáo sư đại học. Ông đã yêu cầu sinh viên của mình thực hiện các hồ sơ cá nhân của 200 cậu bé sống ở khu ổ chuột Baltimore và viết đánh giá về tương lai của từng cậu bé. Kết quả, tất cả sinh viên đều cho rằng những cậu bé này "không có tương lai".

Hai mươi lăm năm sau, một giáo sư khác đã tìm thấy nghiên cứu này và yêu cầu sinh viên của mình theo dõi số phận của những cậu bé đó. Kết quả thật bất ngờ: trong số 180 cậu bé còn lại mà họ tìm được, có đến 176 người đã đạt được thành công vượt trội trong cuộc sống, trở thành luật sư, bác sĩ và doanh nhân.

Khi được hỏi về bí quyết thành công của mình, tất cả những người đàn ông này đều trả lời rằng "Có một người giáo viên." Khi tìm gặp người giáo viên ấy, bà đã chia sẻ rằng bí quyết của mình rất đơn giản: "Tôi đã tin tưởng vào những cậu bé đó."

Nội dung này nhấn mạnh sức mạnh của tình yêu thương và niềm tin, cho thấy rằng tình yêu và sự tin tưởng

In [36]:
print(ask_bot("tóm tắt nội dung phần Love: The one creative force"))

Phần "Love: The one creative force" nhấn mạnh rằng tình yêu thương là sức mạnh duy nhất có khả năng biến đổi kẻ thù thành bạn bè. Đây là một ý tưởng được Martin Luther đề cập, cho thấy tầm quan trọng và sức mạnh của tình yêu trong việc xây dựng và thay đổi các mối quan hệ. Tình yêu không chỉ có thể hàn gắn những vết thương trong tâm hồn mà còn có thể mang lại sự thay đổi tích cực trong cuộc sống của con người.
